In [2]:
import torch
if torch.cuda.is_available():
    import cudf as df
    import cupy as py
else:
    import pandas as df
    import numpy as py
import pandas as pd
from pymilvus import (
    db,
    connections,
    utility,
    Collection,
)
import os
import glob
import cudf
import dask_cudf
import hydra
import logging
import sys
sys.path.append('../../..')
from aiagents4pharma.talk2knowledgegraphs.utils.embeddings.ollama import EmbeddingWithOllama
from aiagents4pharma.talk2knowledgegraphs.utils.extractions.milvus_multimodal_pcst import MultimodalPCSTPruning as MultimodalPCSTPruningNew
from aiagents4pharma.talk2knowledgegraphs.utils.extractions.multimodal_pcst import MultimodalPCSTPruning as MultimodalPCSTPruningOld

/root/anaconda3/envs/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Configuration for Milvus
milvus_host = "localhost"
milvus_port = "19530"
milvus_uri = "http://localhost:19530"
milvus_token = "root:Milvus"
milvus_user = "root"
milvus_password = "Milvus"
milvus_database = "t2kg_primekg"

In [4]:
# Connect to Milvus
connections.connect(
    alias="default",
    host=milvus_host,
    port=milvus_port,
    user=milvus_user,
    password=milvus_password
)

In [5]:
db.using_database(milvus_database)

utility.list_collections()

['t2kg_primekg_edges',
 't2kg_primekg_nodes',
 't2kg_primekg_nodes_biological_process',
 't2kg_primekg_nodes_cellular_component',
 't2kg_primekg_nodes_disease',
 't2kg_primekg_nodes_drug',
 't2kg_primekg_nodes_gene_protein',
 't2kg_primekg_nodes_molecular_function']

In [16]:
colls = {
    "nodes": Collection(f"{milvus_database}_nodes"),
    "edges": Collection(f"{milvus_database}_edges")}
colls["nodes"].load()
colls["edges"].load()


In [7]:
# Get actual triplet_index range
min_id = colls["edges"].query(expr="", output_fields=["triplet_index"], limit=1, order_by="triplet_index")[0]["triplet_index"]
max_id = colls["edges"].query(expr="", output_fields=["triplet_index"], limit=1, order_by="-triplet_index")[0]["triplet_index"]

In [9]:
min_id, max_id

(0, 0)

In [12]:
batch_size = 1_000_000
head_list = []
tail_list = []
for start in range(0, colls["edges"].num_entities, batch_size):
    end = min(start + batch_size, colls["edges"].num_entities)
    print(f"Processing triplet_index range: {start} to {end}")
    batch = colls["edges"].query(
        expr=f"triplet_index >= {start} and triplet_index < {end}",
        output_fields=["head_index", "tail_index"],
    )
    head_list.extend([r["head_index"] for r in batch])
    tail_list.extend([r["tail_index"] for r in batch])
edge_index = py.array([head_list, tail_list])

Processing triplet_index range: 0 to 1000000
Processing triplet_index range: 1000000 to 2000000
Processing triplet_index range: 2000000 to 3000000
Processing triplet_index range: 3000000 to 3904610


In [18]:
edge_index

array([[    0,     1,     2, ..., 35166, 67928, 37608],
       [ 8816,  2787,  5610, ..., 49927, 50777, 49939]])

In [15]:
colls["edges"].query(
        expr=f"triplet_index == 0",
        output_fields=["head_index", "tail_index", "feat"],
    )

data: ["{'head_index': 0, 'tail_index': 8816, 'feat': 'PHYHIP (gene/protein) has a direct relationship of protein_protein:ppi with KIF15 (gene/protein).', 'triplet_index': 0}"]

In [17]:
colls["nodes"].query(
        expr=f"node_index == 8816",
        output_fields=["node_name", "node_type", "desc"],
    )

data: ["{'node_name': 'KIF15', 'node_type': 'gene/protein', 'desc': 'KIF15 belongs to gene/protein node. KIF15 is kinesin family member 15. Predicted to enable microtubule binding activity and microtubule motor activity. Predicted to be involved in microtubule-based movement. Located in membrane. [provided by Alliance of Genome Resources, Apr 2022]', 'node_index': 8816}"]

In [14]:
batch

data: ["{'head_index': 5754, 'tail_index': 33377, 'triplet_index': 3000000}", "{'head_index': 2614, 'tail_index': 33377, 'triplet_index': 3000001}", "{'head_index': 49773, 'tail_index': 33377, 'triplet_index': 3000002}", "{'head_index': 11359, 'tail_index': 41821, 'triplet_index': 3000003}", "{'head_index': 1895, 'tail_index': 41821, 'triplet_index': 3000004}", "{'head_index': 9059, 'tail_index': 41821, 'triplet_index': 3000005}", "{'head_index': 11155, 'tail_index': 41821, 'triplet_index': 3000006}", "{'head_index': 11359, 'tail_index': 66960, 'triplet_index': 3000007}", "{'head_index': 9059, 'tail_index': 66960, 'triplet_index': 3000008}", "{'head_index': 11155, 'tail_index': 66960, 'triplet_index': 3000009}"] ...

In [13]:
edge_index

array([[    0,     1,     2, ..., 35166, 67928, 37608],
       [ 8816,  2787,  5610, ..., 49927, 50777, 49939]])

In [85]:
# Define the data path
DATA_PATH = "../../../aiagents4pharma/talk2knowledgegraphs/tests/files"

# Define the agent state
state = {
    "selections": {
        "gene/protein": [],
        "molecular_function": [],
        "cellular_component": [],
        "biological_process": [],
        "drug": [],
        "disease": []
    },
    "uploaded_files": [],
    "topk_nodes": 5,
    "topk_edges": 5,
    "dic_source_graph": [
        {
            "name": "BioBridge",
            "kg_pyg_path": f"{DATA_PATH}/biobridge_multimodal_pyg_graph.pkl",
            "kg_text_path": f"{DATA_PATH}/biobridge_multimodal_text_graph.pkl",
        }
    ],
}

In [86]:
# Update state
state["uploaded_files"] = [
    {
        "file_name": "multimodal-analysis_single_gene.xlsx",
        "file_path": f"{DATA_PATH}/multimodal-analysis_single_gene.xlsx",
        "file_type": "multimodal",
        "uploaded_by": "VPEUser",
        "uploaded_timestamp": "2025-05-12 00:00:00",
    }
]

In [87]:
# Initialize logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [139]:
# Initialize dataframes
logger.log(logging.INFO, "Initializing dataframes")
multimodal_df = df.DataFrame({"name": [], "node_type": []})
query_df = []

# Loop over the uploaded files and find multimodal files
logger.log(logging.INFO, "Looping over uploaded files")
for i in range(len(state["uploaded_files"])):
    # Check if multimodal file is uploaded
    if state["uploaded_files"][i]["file_type"] == "multimodal":
        # Read the Excel file
        multimodal_df = pd.read_excel(state["uploaded_files"][i]["file_path"],
                                        sheet_name=None)
        
        
# Check if the multimodal_df is empty
logger.log(logging.INFO, "Checking if multimodal_df is empty")
if len(multimodal_df) > 0:
    # Prepare multimodal_df
    logger.log(logging.INFO, "Preparing multimodal_df")
    # Merge all obtained dataframes into a single dataframe
    multimodal_df = pd.concat(multimodal_df).reset_index()
    multimodal_df = df.DataFrame(multimodal_df)
    multimodal_df.drop(columns=["level_1"], inplace=True)
    multimodal_df.rename(columns={"level_0": "q_node_type",
                                  "name": "q_node_name"}, inplace=True)
    # Since an excel sheet name could not contain a `/`,
    # but the node type can be 'gene/protein' as exists in the PrimeKG
    multimodal_df["q_node_type"] = multimodal_df["q_node_type"].str.replace('-', '_')
    
    # Query the Milvus database for each node type in multimodal_df
    logger.log(logging.INFO, "Querying Milvus database for each node type in multimodal_df")
    for node_type, node_type_df in multimodal_df.groupby("q_node_type"):
        print(f"Processing node type: {node_type}")
        
        # Load the collection
        collection = Collection(name=f"{milvus_database}_nodes_{node_type.replace('/', '_')}")
        collection.load()

        # Query the collection with node names from multimodal_df
        q_node_names =  getattr(node_type_df['q_node_name'], 
                                "to_pandas", 
                                lambda: node_type_df['q_node_name'])().tolist()
        q_columns = ["node_id", "node_type", "feat", "feat_emb", "desc", "desc_emb"]
        res = collection.query(
            expr=f'node_name IN [{','.join(f'"{name}"' for name in q_node_names)}]',
            output_fields=q_columns,
        )
        # Convert the embeedings into floats
        for r_ in res:
            r_['feat_emb'] = [float(x) for x in r_['feat_emb']]
            r_['desc_emb'] = [float(x) for x in r_['desc_emb']]

        # Convert the result to a DataFrame
        res_df = df.DataFrame(res)[q_columns]
        res_df["use_description"] = False
        
        # Append the results to query_df
        query_df.append(res_df)
    
    # Concatenate all results into a single DataFrame
    logger.log(logging.INFO, "Concatenating all results into a single DataFrame")
    query_df = df.concat(query_df, ignore_index=True)
    
    # Update the state by adding the the selected node IDs
    logger.log(logging.INFO, "Updating state with selected node IDs")
    state["selections"] = query_df.to_pandas().groupby(
        "node_type"
    )["node_id"].apply(list).to_dict()


INFO:__main__:Initializing dataframes
INFO:__main__:Looping over uploaded files
INFO:__main__:Checking if multimodal_df is empty
INFO:__main__:Preparing multimodal_df
INFO:__main__:Querying Milvus database for each node type in multimodal_df
INFO:__main__:Concatenating all results into a single DataFrame
INFO:__main__:Updating state with selected node IDs


Processing node type: gene_protein


In [140]:
# Prompt
prompt_text = "List the drugs that target Interleukin-6."
# Embed the prompt using Ollama
prompt = {
    "text": prompt_text,
    "emb": [EmbeddingWithOllama(model_name='nomic-embed-text').embed_query(prompt_text)]
}

INFO:httpx:HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


In [141]:
# Append a user prompt to the query dataframe
# query_df = []
# query_df = df.DataFrame({
#     'node_id': 'user_prompt',
#     'node_type': 'prompt',
#     'feat': prompt["text"],
#     'feat_emb': prompt["emb"],
#     'desc': prompt["text"],
#     'desc_emb': prompt["emb"],
#     'use_description': True  # set to True for user prompt embedding
# })


logger.log(logging.INFO, "Adding user prompt to query dataframe")
query_df = df.concat([
    query_df,
    df.DataFrame({
    'node_id': 'user_prompt',
    'node_type': 'prompt',
    'feat': prompt["text"],
    'feat_emb': prompt["emb"],
    'desc': prompt["text"],
    'desc_emb': prompt["emb"],
    'use_description': True  # set to True for user prompt embedding
})
], ignore_index=True)

INFO:__main__:Adding user prompt to query dataframe


In [116]:
query_df

,node_id,node_type,feat,feat_emb,desc,desc_emb,use_description
0,IL7R_(625),gene/protein,MTILGTTFGMVFSLLQVVSGESGYAQNGDLEDAELDDYSFSCYSQL...,"[0.005123700015246868, 0.005596505478024483, 0...",IL7R belongs to gene/protein node. IL7R is int...,"[0.04506901279091835, 0.008911124430596828, -0...",False
1,TCF7_(5195),gene/protein,MPQLDSGGGGAGGGDDLGAPDELLAFQDEGEEQDDKSRDSAAGPER...,"[0.0020665288902819157, 0.00025825444026850164...",TCF7 belongs to gene/protein node. TCF7 is tra...,"[0.03699759021401405, 0.038098547607660294, -0...",False
2,user_prompt,prompt,List the drugs that target Interleukin-6.,"[-0.020274699, -0.0045663384, -0.18202391, 0.0...",List the drugs that target Interleukin-6.,"[-0.020274699, -0.0045663384, -0.18202391, 0.0...",True


In [117]:
# extract_subgraph
# text_emb = py.array(query_df.loc[0]['desc_emb'][0]).reshape(1, -1).astype(py.float32)
# query_emb = py.array(query_df.loc[0]['feat_emb'][0]).reshape(1, -1).astype(py.float32)
i = 0
text_emb = query_df.loc[i]['desc_emb'][i]
query_emb = query_df.loc[i]['feat_emb'][i]
modality = query_df.loc[i]['node_type'][i]
use_description = query_df.loc[i]['use_description'][i]

In [118]:
use_description, modality

(False, 'gene/protein')

In [119]:
# Initialize variables
topk = 3
topk_e = 3
c_const = 0.01

collection = Collection(name=f"{milvus_database}_nodes")
collection.load()
topk = min(topk, collection.num_entities)

n_prizes = df.Series(py.zeros(collection.num_entities), dtype=py.float32)

### Node Prize

In [120]:
# Intialize several variables
collection = Collection(name=f"{milvus_database}_nodes")
collection.load()
topk = min(topk, collection.num_entities)
n_prizes = py.zeros(collection.num_entities, dtype=py.float32)

# Calculate cosine similarity for text features and update the score
if use_description:
    # Search the collection with the query embedding
    res = collection.search(
        data=[query_emb],
        anns_field="desc_emb",
        param={"metric_type": "IP"},
        limit=topk,
        output_fields=["node_id"])
else:
    # Load the collection for the specific node type
    col_name = f"{milvus_database}_nodes_{modality.replace('/', '_')}"
    print(f"Loading collection: {col_name}")
    collection = Collection(name=col_name)
    collection.load()

    # Search the collection with the query embedding
    res = collection.search(
        data=[query_emb],
        anns_field="feat_emb",
        param={"metric_type": "IP"},
        limit=topk,
        output_fields=["node_id"])

# Update the prizes based on the search results
n_prizes[[r['node_index'] for r in res[0]]] = py.arange(topk, 0, -1).astype(py.float32)

Loading collection: t2kg_primekg_nodes_gene_protein


In [121]:
[r['distance'] for r in res[0]]

[0.9999999403953552, 0.9898543357849121, 0.9895499348640442]

In [122]:
[r.id for r in res[0]]

[5, 845, 12]

In [123]:
[r["node_index"] for r in res[0]]

[5, 845, 12]

### Edge Prize

In [124]:
# Intialize several variables
collection = Collection(name=f"{milvus_database}_edges")
collection.load()
topk_e = min(topk_e, collection.num_entities)
e_prizes = py.zeros(collection.num_entities, dtype=py.float32)

# Search the collection with the query embedding
res = collection.search(
    data=[text_emb],
    anns_field="feat_emb",
    param={"metric_type": "IP"},
    limit=topk_e,
    output_fields=["head_id", "tail_id"])

# Update the prizes based on the search results
e_prizes[[r['triplet_index'] for r in res[0]]] = [r['distance'] for r in res[0]]

In [125]:
unique_prizes, inverse_indices = py.unique(e_prizes, return_inverse=True)
topk_e_values = unique_prizes[py.argsort(-unique_prizes)[:topk_e]]
last_topk_e_value = topk_e
for k in range(topk_e):
    indices = inverse_indices == (unique_prizes == topk_e_values[k]).nonzero()[0]
    value = min((topk_e - k) / indices.sum().item(), last_topk_e_value)
    e_prizes[indices] = value
    last_topk_e_value = value * (1 - c_const)

### Create Index

In [126]:
# Intialize several variables
collection = Collection(name=f"{milvus_database}_edges")
collection.load()

edges = collection.query(
    expr="triplet_index >= 0",
    output_fields=["head_index", "tail_index"]
)
edge_index = [
    [r['head_index'] for r in edges],
    [r['tail_index'] for r in edges]
]
edge_index = py.array(edge_index)

In [127]:
edge_index

array([[ 123,   99,  320, ..., 2509, 2304, 2142],
       [  36,   47,   47, ...,   22,  838,  846]])

### Run Subgraph Extraction

In [ ]:
# query_df = df.read_parquet("../../../aiagents4pharma/talk2knowledgegraphs/tests/files/query_df.parquet")

In [129]:
query_df

,node_id,node_type,feat,feat_emb,desc,desc_emb,use_description
0,user_prompt,prompt,List the drugs that target Interleukin-6.,"[-0.020274699, -0.0045663384, -0.18202391, 0.0...",List the drugs that target Interleukin-6.,"[-0.020274699, -0.0045663384, -0.18202391, 0.0...",True


In [130]:
from types import SimpleNamespace

top = 10
top_k_e = 10
cost_e = 0.5
c_const = 0.01
root = -1
num_clusters = 1
pruning = "gw"
verbosity_level = 0
use_description = False
cfg_fe = SimpleNamespace(**{
    "milvus_db": SimpleNamespace(**{
        "alias": "default",
        "host": "localhost",
        "port": "19530",
        "uri": "http://localhost:19530",
        "token": "root:Milvus",
        "user": "root",
        "password": "Milvus",
        "database_name": "t2kg_primekg",
        "collection_edges": "t2kg_primekg_edges",
        "collection_nodes": "t2kg_primekg_nodes",
        "collection_nodes_gene_protein": "t2kg_primekg_nodes_gene_protein",
        "collection_nodes_molecular_function": "t2kg_primekg_nodes_molecular_function",
        "collection_nodes_cellular_component": "t2kg_primekg_nodes_cellular_component",
        "collection_nodes_biological_process": "t2kg_primekg_nodes_biological_process",
        "collection_nodes_drug": "t2kg_primekg_nodes_drug",
        "collection_nodes_disease": "t2kg_primekg_nodes_disease"
    })
})

In [131]:
for q in getattr(query_df, "to_pandas", lambda: query_df)().iterrows():
    print(q[1]['node_type'])

prompt


In [132]:
 import pickle
 with open("../../../aiagents4pharma/talk2knowledgegraphs/tests/files/biobridge_multimodal_pyg_graph.pkl", "rb") as f:
     pyg_graph= pickle.load(f)

In [133]:
# Initialize the subgraph dictionary
subgraphs = []
unified_subgraph = {
    "nodes": [],
    "edges": []
}
# subgraphs = {}
# subgraphs["nodes"] = []
# subgraphs["edges"] = []

# Loop over query embeddings and modalities
for q in getattr(query_df, "to_pandas", lambda: query_df)().iterrows():
    logger.log(logging.INFO, f"Processing query {q[1]['node_id']}")
    # Prepare the PCSTPruning object and extract the subgraph
    # Parameters were set in the configuration file obtained from Hydra
    subgraph = MultimodalPCSTPruningOld(
        topk=state["topk_nodes"],
        topk_e=state["topk_edges"],
        cost_e=cost_e,
        c_const=c_const,
        root=root,
        num_clusters=num_clusters,
        pruning=pruning,
        verbosity_level=verbosity_level,
        use_description=q[1]['use_description'],
    ).extract_subgraph(pyg_graph,
                       torch.tensor(q[1]['desc_emb']), # description embedding
                       torch.tensor(q[1]['feat_emb']), # modal-specific embedding
                       q[1]['node_type'])

    # Append the extracted subgraph to the dictionary
    unified_subgraph["nodes"].append(subgraph["nodes"].tolist())
    unified_subgraph["edges"].append(subgraph["edges"].tolist())
    subgraphs.append((q[1]['node_id'],
                        subgraph["nodes"].tolist(),
                        subgraph["edges"].tolist()))

# Concatenate and get unique node and edge indices
unified_subgraph["nodes"] = py.unique(
    py.concatenate([py.array(list_) for list_ in unified_subgraph["nodes"]])
).tolist()
unified_subgraph["edges"] = py.unique(
    py.concatenate([py.array(list_) for list_ in unified_subgraph["edges"]])
).tolist()

# Convert the unified subgraph and subgraphs to cudf DataFrames
unified_subgraph = df.DataFrame([("Unified Subgraph",
                                    unified_subgraph["nodes"],
                                    unified_subgraph["edges"])],
                                    columns=["name", "nodes", "edges"])
subgraphs = df.DataFrame(subgraphs, columns=["name", "nodes", "edges"])

# Concate both DataFrames
subgraphs = df.concat([unified_subgraph, subgraphs], ignore_index=True)

INFO:__main__:Processing query user_prompt


In [135]:
for sub in getattr(subgraphs, "to_pandas", lambda: subgraphs)().itertuples(index=False):
    print(sub.name)
    print(sub.nodes)
    print(sub.edges)

Unified Subgraph
[8, 14, 51, 547, 743, 749, 816, 1590, 2484, 2531, 2756, 2903]
[532, 551, 1107, 1154, 1159, 3844, 3851, 5871, 6107, 6116, 10384, 10844, 11135]
user_prompt
[8, 14, 51, 547, 743, 749, 816, 1590, 2484, 2531, 2756, 2903]
[3844, 1107, 3851, 532, 551, 11135, 10844, 10384, 1154, 1159, 5871, 6107, 6116]


In [136]:
pyg_graph.triplet_index[sub.edges].unique()

tensor([  182,   186,   187,  4351,  4352,  4752,  4753,  5818,  5823,  5959,
         7416,  8147, 10433])

In [142]:
# Initialize the subgraph dictionary
subgraphs = []
unified_subgraph = {
    "nodes": [],
    "edges": []
}
# subgraphs = {}
# subgraphs["nodes"] = []
# subgraphs["edges"] = []

# Loop over query embeddings and modalities
for q in getattr(query_df, "to_pandas", lambda: query_df)().iterrows():
    logger.log(logging.INFO, f"Processing query {q[1]['node_id']}")
    # Prepare the PCSTPruning object and extract the subgraph
    # Parameters were set in the configuration file obtained from Hydra
    subgraph = MultimodalPCSTPruningNew(
        topk=state["topk_nodes"],
        topk_e=state["topk_edges"],
        cost_e=cost_e,
        c_const=c_const,
        root=root,
        num_clusters=num_clusters,
        pruning=pruning,
        verbosity_level=verbosity_level,
        use_description=q[1]['use_description'],
    ).extract_subgraph(q[1]['desc_emb'],
                       q[1]['feat_emb'],
                       q[1]['node_type'],
                       cfg_fe)

    # Append the extracted subgraph to the dictionary
    unified_subgraph["nodes"].append(subgraph["nodes"].tolist())
    unified_subgraph["edges"].append(subgraph["edges"].tolist())
    subgraphs.append((q[1]['node_id'],
                        subgraph["nodes"].tolist(),
                        subgraph["edges"].tolist()))

# Concatenate and get unique node and edge indices
unified_subgraph["nodes"] = py.unique(
    py.concatenate([py.array(list_) for list_ in unified_subgraph["nodes"]])
).tolist()
unified_subgraph["edges"] = py.unique(
    py.concatenate([py.array(list_) for list_ in unified_subgraph["edges"]])
).tolist()

# Convert the unified subgraph and subgraphs to cudf DataFrames
unified_subgraph = df.DataFrame([("Unified Subgraph",
                                    unified_subgraph["nodes"],
                                    unified_subgraph["edges"])],
                                    columns=["name", "nodes", "edges"])
subgraphs = df.DataFrame(subgraphs, columns=["name", "nodes", "edges"])

# Concate both DataFrames
subgraphs = df.concat([unified_subgraph, subgraphs], ignore_index=True)

INFO:__main__:Processing query IL7R_(625)
INFO:aiagents4pharma.talk2knowledgegraphs.utils.extractions.milvus_multimodal_pcst:compute_prizes
INFO:aiagents4pharma.talk2knowledgegraphs.utils.extractions.milvus_multimodal_pcst:_compute_node_prizes
INFO:aiagents4pharma.talk2knowledgegraphs.utils.extractions.milvus_multimodal_pcst:_compute_edge_prizes
INFO:aiagents4pharma.talk2knowledgegraphs.utils.extractions.milvus_multimodal_pcst:Creating edge index
INFO:aiagents4pharma.talk2knowledgegraphs.utils.extractions.milvus_multimodal_pcst:compute_subgraph_costs
INFO:aiagents4pharma.talk2knowledgegraphs.utils.extractions.milvus_multimodal_pcst:Running PCST algorithm
INFO:aiagents4pharma.talk2knowledgegraphs.utils.extractions.milvus_multimodal_pcst:Getting subgraph nodes and edges
INFO:__main__:Processing query TCF7_(5195)
INFO:aiagents4pharma.talk2knowledgegraphs.utils.extractions.milvus_multimodal_pcst:compute_prizes
INFO:aiagents4pharma.talk2knowledgegraphs.utils.extractions.milvus_multimodal_pc

In [143]:
for sub in getattr(subgraphs, "to_pandas", lambda: subgraphs)().itertuples(index=False):
    print(sub.name)
    print(sub.nodes)
    print(sub.edges)

Unified Subgraph
[5, 8, 12, 14, 22, 27, 30, 54, 72, 82, 547, 743, 749, 816, 840, 845, 846, 855, 1254, 1320, 1415, 1863, 1952, 2102, 2263, 2479, 2484, 2531, 2756, 2779, 2827, 2903]
[182, 186, 187, 323, 888, 920, 921, 942, 1004, 1083, 1191, 1216, 1268, 1650, 1760, 1846, 2486, 4508, 4509, 4557, 4558, 4753, 4958, 5110, 5818, 5823, 6613, 6762, 7397, 7400, 7416, 8147, 10433]
IL7R_(625)
[5, 12, 22, 82, 840, 845, 855, 1952, 2479, 2827]
[7400, 6613, 6762, 7397, 920, 921, 1760, 1846, 4958]
TCF7_(5195)
[27, 30, 54, 72, 840, 846, 855, 1320, 1415, 1863, 2102, 2263, 2779]
[1268, 1216, 1083, 1004, 1191, 888, 942, 1650, 2486, 4557, 4558, 5110]
user_prompt
[8, 14, 547, 743, 749, 816, 1254, 2484, 2531, 2756, 2903]
[4509, 4508, 4753, 323, 8147, 7416, 10433, 182, 186, 187, 5818, 5823]


In [33]:
coll_name = f"{milvus_database}_nodes"
node_coll = Collection(name=coll_name)
node_coll.load()
nodes = node_coll.query(
    expr=f'node_index IN [{",".join(f"{n}" for n in [8, 14, 547, 743, 749, 816, 1254, 2484, 2531, 2756, 2903])}]',
    output_fields=['node_id', 'node_name', 'node_type', 'desc']
)
nodes = df.DataFrame(nodes)
nodes

,node_id,node_name,node_type,desc,node_index
0,RELA_(772),RELA,gene/protein,RELA belongs to gene/protein node. RELA is REL...,8
1,IL6_(1567),IL6,gene/protein,IL6 belongs to gene/protein node. IL6 is inter...,14
2,Binimetinib_(15391),Binimetinib,drug,"Binimetinib belongs to drug node. Binimetinib,...",547
3,Atiprimod_(17591),Atiprimod,drug,Atiprimod belongs to drug node. Investigat...,743
4,Dilmapimod_(17602),Dilmapimod,drug,Dilmapimod belongs to drug node. Dilmapimod ha...,749
5,SC-236_(20186),SC-236,drug,SC-236 belongs to drug node. SC-236 is a poten...,816
6,regulation of transcription initiation from RN...,regulation of transcription initiation from RN...,biological_process,regulation of transcription initiation from RN...,1254
7,interleukin-6-mediated signaling pathway_(108960),interleukin-6-mediated signaling pathway,biological_process,interleukin-6-mediated signaling pathway belon...,2484
8,cellular response to interleukin-6_(109554),cellular response to interleukin-6,biological_process,cellular response to interleukin-6 belongs to ...,2531
9,interleukin-6 receptor binding_(117413),interleukin-6 receptor binding,molecular_function,interleukin-6 receptor binding belongs to mole...,2756


In [34]:
coll_name = f"{milvus_database}_edges"
edge_coll = Collection(name=coll_name)
edge_coll.load()
graph_edges = edge_coll.query(
    expr=f'triplet_index IN [{",".join(f"{e}" for e in [182, 186, 187, 323, 4508, 4509, 4753, 5818, 5823, 7416, 8147, 10433])}]',
    output_fields=['head_id', 'tail_id', 'edge_type']
)
graph_edges = df.DataFrame(graph_edges)
graph_edges

,head_id,tail_id,edge_type,triplet_index
0,Atiprimod_(17591),IL6_(1567),drug|target|gene/protein,182
1,Binimetinib_(15391),IL6_(1567),drug|target|gene/protein,186
2,Dilmapimod_(17602),IL6_(1567),drug|target|gene/protein,187
3,SC-236_(20186),RELA_(772),drug|target|gene/protein,323
4,IL6_(1567),regulation of transcription initiation from RN...,gene/protein|interacts with|biological_process,4508
5,RELA_(772),regulation of transcription initiation from RN...,gene/protein|interacts with|biological_process,4509
6,RELA_(772),cellular response to interleukin-6_(109554),gene/protein|interacts with|biological_process,4753
7,IL6_(1567),Atiprimod_(17591),gene/protein|target|drug,5818
8,IL6_(1567),Dilmapimod_(17602),gene/protein|target|drug,5823
9,interleukin-6 receptor binding_(117413),IL6_(1567),molecular_function|interacts with|gene/protein,7416


### Optimization

In [20]:
import pickle
local_dir = "../../../aiagents4pharma/talk2knowledgegraphs/tests/files"
# Input
with open(f"{local_dir}/input_edge_index.pkl", "rb") as f:
    edge_index = pickle.load(f)
with open(f"{local_dir}/input_prizes.pkl", "rb") as f:
    prizes = pickle.load(f)
num_nodes = 2991
# # Output
# with open(f"{local_dir}/output_edges_dict.pkl", "rb") as f:
#     output_edges_dict = pickle.load(f)
# with open(f"{local_dir}/output_prizes.pkl", "rb") as f:
#     output_prizes = pickle.load(f)
# with open(f"{local_dir}/output_costs.pkl", "rb") as f:
#     output_costs = pickle.load(f)
# with open(f"{local_dir}/output_mapping.pkl", "rb") as f:
#     output_mapping = pickle.load(f)

In [26]:
import logging
# Initialize logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

cost_e = 0.5
c_const = 0.01

In [ ]:
# Initialize several variables
real_ = {}
virt_ = {}
edge_index = py.array(edge_index)

# Update edge cost threshold
updated_cost_e = min(
    cost_e,
    py.max(prizes["edges"]).item() * (1 - c_const / 2),
)

# Masks for real and virtual edges
logger.log(logging.INFO, "Creating masks for real and virtual edges")
real_["mask"] = prizes["edges"] <= updated_cost_e
virt_["mask"] = ~real_["mask"]

# Real edge indices
logger.log(logging.INFO, "Computing real edges")
real_["indices"] = py.nonzero(real_["mask"])[0]
real_["src"] = edge_index[0][real_["indices"]]
real_["dst"] = edge_index[1][real_["indices"]]
real_["edges"] = py.stack([real_["src"], real_["dst"]], axis=1)
real_["costs"] = updated_cost_e - prizes["edges"][real_["indices"]]

# Edge index mapping: local real edge idx -> original global index
logger.log(logging.INFO, "Creating mapping for real edges")
# mapping_edges = {int(i): int(j) for i, j in enumerate(real_["indices"])}
mapping_edges = dict(zip(range(len(real_["indices"])), real_["indices"].tolist()))

# Virtual edge handling
logger.log(logging.INFO, "Computing virtual edges")
virt_["indices"] = py.nonzero(virt_["mask"])[0]
virt_["src"] = edge_index[0][virt_["indices"]]
virt_["dst"] = edge_index[1][virt_["indices"]]
virt_["prizes"] = prizes["edges"][virt_["indices"]] - updated_cost_e

# Generate virtual node IDs
logger.log(logging.INFO, "Generating virtual node IDs")
virt_["num"] = virt_["indices"].shape[0]
virt_["node_ids"] = py.arange(num_nodes, num_nodes + virt_["num"])

# Virtual edges: (src → virtual), (virtual → dst)
logger.log(logging.INFO, "Creating virtual edges")
virt_["edges_1"] = py.stack([virt_["src"], virt_["node_ids"]], axis=1)
virt_["edges_2"] = py.stack([virt_["node_ids"], virt_["dst"]], axis=1)
virt_["edges"] = py.concatenate([virt_["edges_1"],
                                    virt_["edges_2"]], axis=0)
virt_["costs"] = py.zeros((virt_["edges"].shape[0],), dtype=real_["costs"].dtype)

# Combine real and virtual edges/costs
logger.log(logging.INFO, "Combining real and virtual edges/costs")
all_edges = py.concatenate([real_["edges"], virt_["edges"]], axis=0)
all_costs = py.concatenate([real_["costs"], virt_["costs"]], axis=0)

# Final prizes
logger.log(logging.INFO, "Getting final prizes")
final_prizes = py.concatenate([prizes["nodes"], virt_["prizes"]], axis=0)

# Mapping virtual node ID -> edge index in original graph
logger.log(logging.INFO, "Creating mapping for virtual nodes")
mapping_nodes = dict(zip(virt_["node_ids"].tolist(), virt_["indices"].tolist()))

# Build return values
logger.log(logging.INFO, "Building return values")
edges_dict = {
    "edges": all_edges,
    "num_prior_edges": real_["edges"].shape[0],
}
mapping = {
    "edges": mapping_edges,
    "nodes": mapping_nodes,
}


INFO:__main__:Creating masks for real and virtual edges
INFO:__main__:Computing real edges
INFO:__main__:Computing real edges
INFO:__main__:Creating mapping for real edges
INFO:__main__:Creating mapping for real edges
INFO:__main__:Computing virtual edges
INFO:__main__:Generating virtual node IDs
INFO:__main__:Creating virtual edges
INFO:__main__:Combining real and virtual edges/costs
INFO:__main__:Getting final prizes
INFO:__main__:Computing virtual edges
INFO:__main__:Generating virtual node IDs
INFO:__main__:Creating virtual edges
INFO:__main__:Combining real and virtual edges/costs
INFO:__main__:Getting final prizes
INFO:__main__:Creating mapping for virtual nodes
INFO:__main__:Building return values
INFO:__main__:Creating mapping for virtual nodes
INFO:__main__:Building return values


In [53]:
{int(nid): int(idx) for nid, idx in zip(virt_["node_ids"],
                                                        virt_["indices"])}

{2991: 325862, 2992: 334087, 2993: 3395165, 2994: 3403643, 2995: 3403644}

In [54]:
mapping_nodes

{2991: 325862, 2992: 334087, 2993: 3395165, 2994: 3403643, 2995: 3403644}

In [49]:
mapping_nodes

{2991: 325862, 2992: 334087, 2993: 3395165, 2994: 3403643, 2995: 3403644}

In [55]:
mapping_edges

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 98,
 99: 99,
 100: 100,
 101: 101,
 102: 102,
 103: 103,
 104: 104,
 105: 105,
 106: 106,
 107: 107,
 108: 108,
 109: 109,
 110: 110,

In [1]:
coll = Collection(name=f"{milvus_database}_nodes_gene_protein")
coll.load()

NameError: name 'Collection' is not defined

In [61]:
coll.query(
    expr="node_index == 0",
    output_fields=["node_id", "node_name", "node_type", "feat"]
)

data: ["{'node_id': 'PHYHIP_(0)', 'node_name': 'PHYHIP', 'node_type': 'gene/protein', 'feat': 'MELLSTPHSIEINNITCDSFRISWAMEDSDLERVTHYFIDLNKKENKNSNKFKHRDVPTKLVAKAVPLPMTVRGHWFLSPRTEYSVAVQTAVKQSDGEYLVSGWSETVEFCTGDYAKEHLAQLQEKAEQIAGRMLRFSVFYRNHHKEYFQHARTHCGNMLQPYLKDNSGSHGSPTSGMLHGVFFSCNTEFNTGQPPQDSPYGRWRFQIPAQRLFNPSTNLYFADFYCMYTAYHYAILVLAPKGSLGDRFCRDRLPLLDIACNKFLTCSVEDGELVFRHAQDLILEIIYTEPVDLSLGTLGEISGHQLMSLSTADAKKDPSCKTCNISVGR', 'node_index': 0}"]

In [56]:
mapping_edges[3904604]

3904609

In [57]:
len(list(mapping_edges.keys()))

3904605

In [46]:
mapping_edges = {int(i): int(j) for i, j in enumerate(real_["indices"])}
mapping_edges_new = dict(zip(range(len(real_["indices"])), real_["indices"].tolist()))

In [29]:
real_["indices"]

array([      0,       1,       2, ..., 3904607, 3904608, 3904609])

In [ ]:
{int(i): int(j) for i, j in enumerate(real_["indices"])}

In [151]:
edge_index

array([[ 123,   99,  320, ..., 2509, 2304, 2142],
       [  36,   47,   47, ...,   22,  838,  846]])

In [156]:
py.unique(prizes["nodes"])

array([0., 1., 2., 3., 4., 5.], dtype=float32)

In [157]:
py.unique(prizes["edges"])

array([0., 1., 2., 3., 4., 5.], dtype=float32)

In [158]:
updated_cost_e = min(
    cost_e,
    prizes["edges"].max().item() * (1 - c_const / 2),
)
updated_cost_e


0.5

In [165]:
# Initialize variables
edges = []
costs = []
virtual = {
    "n_prizes": [],
    "edges": [],
    "costs": [],
}
mapping = {"nodes": {}, "edges": {}}

In [ ]:
# Logic to reduce the cost of the edges such that at least one edge is selected
updated_cost_e = min(
    cost_e,
    prizes["edges"].max().item() * (1 - self.c_const / 2),
)

# Initialize variables
edges = []
costs = []
virtual = {
    "n_prizes": [],
    "edges": [],
    "costs": [],
}
mapping = {"nodes": {}, "edges": {}}

# Compute the costs, edges, and virtual variables based on the prizes
for i, (src, dst) in enumerate(edge_index.T):
    prize_e = prizes["edges"][i].item()
    if prize_e <= updated_cost_e:
        mapping["edges"][len(edges)] = i
        edges.append((src.item(), dst.item()))
        costs.append(updated_cost_e - prize_e)
    else:
        virtual_node_id = num_nodes + len(virtual["n_prizes"])
        mapping["nodes"][virtual_node_id] = i
        virtual["edges"].append((src.item(), virtual_node_id))
        virtual["edges"].append((virtual_node_id, dst.item()))
        virtual["costs"].append(0)
        virtual["costs"].append(0)
        virtual["n_prizes"].append(prize_e - updated_cost_e)
prizes = py.concatenate([prizes["nodes"], py.array(virtual["n_prizes"])])
edges_dict = {}
edges_dict["edges"] = edges
edges_dict["num_prior_edges"] = len(edges)
# Final computation of the costs and edges based on the virtual costs and virtual edges
if len(virtual["costs"]) > 0:
    costs = py.array(costs + virtual["costs"])
    edges = py.array(edges + virtual["edges"])
    edges_dict["edges"] = edges

return edges_dict, prizes, costs, mapping

In [172]:
c_const, cost_e

(0.01, 0.5)

In [176]:
isinstance(prizes["edges"], py.ndarray)

True

In [228]:
# Initialize the collections dictionary
colls = {}

# Load the collection for nodes
colls["nodes"] = Collection(name=f"{milvus_database}_nodes")

if modality != "prompt":
    # Load the collection for the specific node type
    colls["nodes_type"] = Collection(
        f"{milvus_database}_nodes_{modality.replace('/', '_')}"
    )

# Load the collection for edges
colls["edges"] = Collection(name=f"{milvus_database}_edges")

# Load the collections
for coll in colls.values():
    coll.load()

In [227]:
colls["nodes"].load()

In [179]:
def compute_subgraph_costs(edge_index, num_nodes, prizes, c_const=0.01, cost_e=0.5):
    src, dst = edge_index  # shape: (2, num_edges)
    prizes_edges = prizes["edges"]
    
    # Update edge cost threshold
    updated_cost_e = min(
        cost_e,
        py.max(prizes_edges).item() * (1 - c_const / 2),
    )

    # Masks for real and virtual edges
    mask_real = prizes_edges <= updated_cost_e
    mask_virtual = ~mask_real

    # Real edge indices
    real_indices = py.nonzero(mask_real)[0]
    real_src = src[real_indices]
    real_dst = dst[real_indices]
    real_edges = py.stack([real_src, real_dst], axis=1)
    real_costs = updated_cost_e - prizes_edges[real_indices]

    # Edge index mapping: local real edge idx -> original global index
    mapping_edges = {int(i): int(j) for i, j in enumerate(real_indices)}

    # Virtual edge handling
    virtual_indices = py.nonzero(mask_virtual)[0]
    virtual_src = src[virtual_indices]
    virtual_dst = dst[virtual_indices]
    virtual_prizes = prizes_edges[virtual_indices] - updated_cost_e

    # Generate virtual node IDs
    num_virtual = virtual_indices.shape[0]
    virtual_node_ids = py.arange(num_nodes, num_nodes + num_virtual)

    # Virtual edges: (src → virtual), (virtual → dst)
    v_edges_1 = py.stack([virtual_src, virtual_node_ids], axis=1)
    v_edges_2 = py.stack([virtual_node_ids, virtual_dst], axis=1)
    virtual_edges = py.concatenate([v_edges_1, v_edges_2], axis=0)
    virtual_costs = py.zeros((virtual_edges.shape[0],), dtype=real_costs.dtype)

    # Combine real and virtual edges/costs
    all_edges = py.concatenate([real_edges, virtual_edges], axis=0)
    all_costs = py.concatenate([real_costs, virtual_costs], axis=0)

    # Final prizes
    final_prizes = py.concatenate([prizes["nodes"], virtual_prizes], axis=0)

    # Mapping virtual node ID -> edge index in original graph
    mapping_nodes = {int(nid): int(idx) for nid, idx in zip(virtual_node_ids, virtual_indices)}

    # Build return values
    edges_dict = {
        "edges": all_edges,
        "num_prior_edges": real_edges.shape[0],
    }
    mapping = {
        "edges": mapping_edges,
        "nodes": mapping_nodes,
    }

    return edges_dict, final_prizes, all_costs, mapping


In [219]:
src, dst = edge_index

In [220]:
src

array([ 123,   99,  320, ..., 2509, 2304, 2142])

In [222]:
dst

array([ 36,  47,  47, ...,  22, 838, 846])

In [221]:
edge_index[0]

array([ 123,   99,  320, ..., 2509, 2304, 2142])

In [223]:
edge_index[1]

array([ 36,  47,  47, ...,  22, 838, 846])

In [218]:
prizes["edges"]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [180]:
outputs = compute_subgraph_costs(
    edge_index,
    num_nodes,
    prizes,
)

In [197]:
py.unique(output_edges_dict["edges"].flatten()).shape

(2993,)

In [208]:
output_prizes.shape, output_costs.shape

((2996,), (11277,))

In [217]:
sum(py.unique(output_edges_dict["edges"].flatten()) == py.unique(outputs[0]['edges'].flatten()))
output_edges_dict['num_prior_edges'] == outputs[0]['num_prior_edges']
output_prizes == outputs[1]
output_costs == outputs[2]
output_mapping['nodes'], outputs[3]['nodes']
for k in output_mapping['edges'].keys():
    print(output_mapping['edges'][k] == outputs[3]['edges'][k])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
output_edges_dict['edges'], final_prizes, all_costs, mapping

({'edges': array([[ 123,   36],
         [  99,   47],
         [ 320,   47],
         ...,
         [2993,  744],
         [2994,  748],
         [2995,  119]]),
  'num_prior_edges': 11267},
 array([0. , 0. , 0. , ..., 4.5, 0.5, 3.5], dtype=float32),
 array([0.5, 0.5, 0.5, ..., 0. , 0. , 0. ], dtype=float32),
 {'edges': {0: 0,
   1: 1,
   2: 2,
   3: 3,
   4: 4,
   5: 5,
   6: 6,
   7: 7,
   8: 8,
   9: 9,
   10: 10,
   11: 11,
   12: 12,
   13: 13,
   14: 14,
   15: 15,
   16: 16,
   17: 17,
   18: 18,
   19: 19,
   20: 20,
   21: 21,
   22: 22,
   23: 23,
   24: 24,
   25: 25,
   26: 26,
   27: 27,
   28: 28,
   29: 29,
   30: 30,
   31: 31,
   32: 32,
   33: 33,
   34: 34,
   35: 35,
   36: 36,
   37: 37,
   38: 38,
   39: 39,
   40: 40,
   41: 41,
   42: 42,
   43: 43,
   44: 44,
   45: 45,
   46: 46,
   47: 47,
   48: 48,
   49: 49,
   50: 50,
   51: 51,
   52: 52,
   53: 53,
   54: 54,
   55: 55,
   56: 56,
   57: 57,
   58: 58,
   59: 59,
   60: 60,
   61: 61,
   62: 62,
   63:

In [149]:
edges_dict, prizes, costs, mapping

({'edges': array([[ 123,   36],
         [  99,   47],
         [ 320,   47],
         ...,
         [2994,  365],
         [  24, 2995],
         [2995,  119]]),
  'num_prior_edges': 11267},
 array([0. , 0. , 0. , ..., 3.5, 1.5, 2.5]),
 array([0.5, 0.5, 0.5, ..., 0. , 0. , 0. ]),
 {'nodes': {2991: 167, 2992: 5801, 2993: 5803, 2994: 5810, 2995: 5814},
  'edges': {0: 0,
   1: 1,
   2: 2,
   3: 3,
   4: 4,
   5: 5,
   6: 6,
   7: 7,
   8: 8,
   9: 9,
   10: 10,
   11: 11,
   12: 12,
   13: 13,
   14: 14,
   15: 15,
   16: 16,
   17: 17,
   18: 18,
   19: 19,
   20: 20,
   21: 21,
   22: 22,
   23: 23,
   24: 24,
   25: 25,
   26: 26,
   27: 27,
   28: 28,
   29: 29,
   30: 30,
   31: 31,
   32: 32,
   33: 33,
   34: 34,
   35: 35,
   36: 36,
   37: 37,
   38: 38,
   39: 39,
   40: 40,
   41: 41,
   42: 42,
   43: 43,
   44: 44,
   45: 45,
   46: 46,
   47: 47,
   48: 48,
   49: 49,
   50: 50,
   51: 51,
   52: 52,
   53: 53,
   54: 54,
   55: 55,
   56: 56,
   57: 57,
   58: 58,
   59: 5

In [ ]:
graph_edges['edge_type'].str.split('|')

In [ ]:
sub.nodes, sub.edges

In [ ]:
sub.edges

In [ ]:
subgraphs

In [ ]:
unified_subgraph

In [ ]:
subgraphs

In [ ]:
n_prizes.tolist()

In [ ]:
# Intialize several variables
collection = Collection(name=f"{milvus_database}_edges")
collection.load()

results = collection.query(
    expr="triplet_index >= 0",
    output_fields=["head_index", "tail_index"],
)

In [ ]:
np.array([[r['head_index'], r['tail_index']] for r in results])

In [ ]:
topk_e

In [ ]:
inverse_indices

In [ ]:
indices = inverse_indices == (unique_prizes == topk_e_values[0]).nonzero()[0]

In [ ]:
py.where(py.array(indices) == True)

In [ ]:
indices.sum().item()

In [ ]:
(topk_e - 0) / indices.sum().item()

In [ ]:
min((topk_e - 0) / indices.sum().item(), last_topk_e_value)

In [ ]:
[r['distance'] for r in res[0]]

In [ ]:
e_prizes[[r['triplet_index'] for r in res[0]]]

In [ ]:
[r['triplet_index'] for r in res[0]]

In [ ]:
[r['head_id'] for r in res[0]]

In [ ]:
[r['tail_id'] for r in res[0]]

In [ ]:
# ["ACTG1", "AIF1", "ALOX5AP", "ANKRD12", "ANXA1", "ANXA5", "APBB1IP", "ARF6", "ARHGDIB", "ARID4B", "ATP5D", "B2M", "BTG1", "C12ORF57", "C12ORF75", "C19ORF60", "C9ORF142", "CALM1", "CAPZB", "CCL5", "CCR7", "CD2", "CD3E", "CD52", "CD74", "CD8A", "CD8B", "CD96", "CEBPD", "CHCHD10", "CKLF", "CMC1", "COA4", "CORO1B", "COX4I1", "CRIP1", "CST7", "CTSW", "CX3CR1", "DDX5", "DSTN", "DUSP1", "EEF1A1", "EEF1G", "EEF2", "EFHD2", "EIF1", "EIF1AX", "EIF3E", "FAM173A", "FGD3", "FGFBP2", "FOS", "FTH1", "FUBP1", "FYB", "GLRX", "GLTSCR2", "GMFG", "GNAS", "GNLY", "GPR183", "GYPC", "GZMA", "GZMB", "GZMH", "GZMK", "GZMM", "H3F3A", "HCST", "HLA-A", "HLA-DPB1", "HMGB2", "HNRNPU", "HOPX", "HSP90B1", "IFITM2", "IL32", "IL7R", "ILF2", "ITGA4", "ITGB1", "ITM2B", "KLF6", "KLRB1", "KLRD1", "KLRF1", "KLRG1", "KLRK1", "KRTCAP2", "LEPROTL1", "LGALS1", "LGALS3", "LIME1", "LIMS1", "LTB", "LYAR", "MALT1", "MBP", "MT-ATP6", "MT-CO1", "MT-CO2", "MT-CO3", "MT-CYB", "MT-ND1", "MT-ND2", "MT-ND3", "MT-ND4", "MT-ND5", "MT-ND6", "MT2A", "MTIF3", "MTRNR2L12", "NCR3", "NDUFA13", "NDUFV2", "NKG7", "NKTR", "NOSIP", "NSA2", "NSG1", "PASK", "PFN1", "PHACTR2", "PHF11", "PHF3", "PLCG2", "PLEK", "PLIN2", "PNISR", "PPIB", "PPP2R5C", "PRF1", "PTPRC", "RNASET2", "RPL13A", "RPL14", "RPL17", "RPL21", "RPL23", "RPL30", "RPL31", "RPL32", "RPL36", "RPL36AL", "RPL37", "RPL38", "RPL39", "RPL41", "RPLP0", "RPLP1", "RPS11", "RPS12", "RPS15A", "RPS18", "RPS19", "RPS21", "RPS24", "RPS27", "RPS29", "RPS3A", "RPS6", "RPS8", "RPSA", "S100A10", "S100A11", "S100A4", "S100A6", "SELL", "SET", "SH2D1A", "SLC25A5", "SLC4A10", "SPOCK2", "SRP14", "TCF7", "TIGIT", "TIMP1", "TMEM66", "TMSB10", "TMSB4X", "TPT1", "TSHZ2", "TXN", "TXNIP", "TYROBP", "UCP2", "VIM", "ZEB2"]

In [11]:
",".join(f"{n}" for n in nodes_list)

'ACTG1,AIF1,ALOX5AP,ANKRD12,ANXA1,ANXA5,APBB1IP,ARF6,ARHGDIB,ARID4B,ATP5D,B2M,BTG1,C12ORF57,C12ORF75,C19ORF60,C9ORF142,CALM1,CAPZB,CCL5,CCR7,CD2,CD3E,CD52,CD74,CD8A,CD8B,CD96,CEBPD,CHCHD10,CKLF,CMC1,COA4,CORO1B,COX4I1,CRIP1,CST7,CTSW,CX3CR1,DDX5,DSTN,DUSP1,EEF1A1,EEF1G,EEF2,EFHD2,EIF1,EIF1AX,EIF3E,FAM173A,FGD3,FGFBP2,FOS,FTH1,FUBP1,FYB,GLRX,GLTSCR2,GMFG,GNAS,GNLY,GPR183,GYPC,GZMA,GZMB,GZMH,GZMK,GZMM,H3F3A,HCST,HLA-A,HLA-DPB1,HMGB2,HNRNPU,HOPX,HSP90B1,IFITM2,IL32,IL7R,ILF2,ITGA4,ITGB1,ITM2B,KLF6,KLRB1,KLRD1,KLRF1,KLRG1,KLRK1,KRTCAP2,LEPROTL1,LGALS1,LGALS3,LIME1,LIMS1,LTB,LYAR,MALT1,MBP,MT-ATP6,MT-CO1,MT-CO2,MT-CO3,MT-CYB,MT-ND1,MT-ND2,MT-ND3,MT-ND4,MT-ND5,MT-ND6,MT2A,MTIF3,MTRNR2L12,NCR3,NDUFA13,NDUFV2,NKG7,NKTR,NOSIP,NSA2,NSG1,PASK,PFN1,PHACTR2,PHF11,PHF3,PLCG2,PLEK,PLIN2,PNISR,PPIB,PPP2R5C,PRF1,PTPRC,RNASET2,RPL13A,RPL14,RPL17,RPL21,RPL23,RPL30,RPL31,RPL32,RPL36,RPL36AL,RPL37,RPL38,RPL39,RPL41,RPLP0,RPLP1,RPS11,RPS12,RPS15A,RPS18,RPS19,RPS21,RPS24,RPS27,RPS29,RPS3A,RPS6,RPS8,RPSA,S100

In [14]:
milvus_db_name

NameError: name 'milvus_db_name' is not defined

In [19]:
# Load the collection
collection = Collection(
    name=f"{milvus_database}_nodes_gene_protein"
)
collection.load()

# Query the collection with node names from multimodal_df
q_node_names =  [
    "ACTG1",
    "AIF1",
    "ALOX5AP",
    "ANKRD12",
    "ANXA1",
    "ANXA5",
    "APBB1IP",
    "ARF6",
    "ARHGDIB",
    "ARID4B",
    "ATP5D",
    "B2M",
    "BTG1",
    "C12ORF57",
    "C12ORF75",
    "C19ORF60",
    "C9ORF142",
    "CALM1",
    "CAPZB"
]
q_columns = ["node_id", "node_type", "feat", "feat_emb", "desc", "desc_emb"]
res = collection.query(
    expr=f'node_name IN [{','.join(f'"{name}"' for name in q_node_names)}]',
    output_fields=q_columns,
)
# Convert the embeedings into floats
for r_ in res:
    r_['feat_emb'] = [float(x) for x in r_['feat_emb']]
    r_['desc_emb'] = [float(x) for x in r_['desc_emb']]

# Convert the result to a DataFrame
res_df = df.DataFrame(res)[q_columns]
res_df["use_description"] = False

res_df

,node_id,node_type,feat,feat_emb,desc,desc_emb,use_description
0,CALM1_(61),gene/protein,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...,"[0.0014321492053568363, 0.011274673044681549, ...",CALM1 belongs to gene/protein node. CALM1 is c...,"[-0.023683598265051842, -0.008636790327727795,...",False
1,ACTG1_(2169),gene/protein,MEEEIAALVIDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVG...,"[-0.004920232575386763, 0.004783452954143286, ...",ACTG1 belongs to gene/protein node. ACTG1 is a...,"[-0.03399347513914108, -0.02692696824669838, -...",False
2,ANXA1_(2319),gene/protein,MAMVSEFLKQAWFIENEEQEYVQTVKSSKGGPGSAVSPYPTFNPSS...,"[-0.005346118472516537, 0.010328142903745174, ...",ANXA1 belongs to gene/protein node. ANXA1 is a...,"[-0.033988308161497116, 0.010062502697110176, ...",False
3,B2M_(2347),gene/protein,MSRSVALAVLALLSLSGLEAIQRTPKIQVYSRHPAENGKSNFLNCY...,"[0.0035890676081180573, 0.0037703460548073053,...",B2M belongs to gene/protein node. B2M is beta-...,"[-0.03881131112575531, -0.008366833440959454, ...",False
4,CAPZB_(2559),gene/protein,MSDQQLDCALDLMRRLPPQQIEKNLSDLIDLVPSLCEDLLSSVDQP...,"[-0.012172527611255646, -0.006136036477982998,...",CAPZB belongs to gene/protein node. CAPZB is c...,"[-0.04837847501039505, -0.02744496427476406, -...",False
5,ARF6_(3626),gene/protein,MGKVLSKIFGNKEMRILMLGLDAAGKTTILYKLKLGQSVTTIPTVG...,"[-0.0018661182839423418, -0.001126629533246159...",ARF6 belongs to gene/protein node. ARF6 is ADP...,"[-0.0254753977060318, -0.010671228170394897, -...",False
6,ANXA5_(5004),gene/protein,MAQVLRGTVTDFPGFDERADAETLRKAMKGLGTDEESILTLLTSRS...,"[0.007284759543836117, 0.010022249072790146, 0...",ANXA5 belongs to gene/protein node. ANXA5 is a...,"[-0.03790163993835449, 0.0032147530000656843, ...",False
7,ARHGDIB_(5936),gene/protein,MTEKAPEPHVEEDDDDELDSKLNYKPPPQKSLKELQEMDKDDESLI...,"[0.00248607131652534, -0.00035060098161920905,...",ARHGDIB belongs to gene/protein node. ARHGDIB ...,"[-0.029910219833254814, -0.017761699855327606,...",False
8,ANKRD12_(6038),gene/protein,MPKSGFTKPIQSENSDSDSNMVEKPYGRKSKDKIASYSKTPKIERS...,"[0.0005237162695266306, -0.0031139254570007324...",ANKRD12 belongs to gene/protein node. ANKRD12 ...,"[-0.03076424077153206, 0.0026955113280564547, ...",False
9,ARID4B_(6869),gene/protein,MKALDEPPYLTVGTDVSAKYRGAFCEAKIKTAKRLVKVKVTFRHDS...,"[-0.0054492587223649025, -0.001639880938455462...",ARID4B belongs to gene/protein node. ARID4B is...,"[-0.05148608982563019, -0.011065568774938583, ...",False


In [ ]:
coll = Collection(name=f"{milvus_database}_nodes")
coll.load()

nodes_list = ["ACTG1", "AIF1", "ALOX5AP"]

graph_nodes = coll.query(
    expr=f'node_name IN [{",".join(f"{n}" for n in nodes_list)}]',
    output_fields=['node_id', 'node_name', 'node_type', 'desc']
)
graph_nodes = df.DataFrame(graph_nodes)



2025-07-09 17:58:25,106 [ERROR][handler]: RPC error: [query], <MilvusException: (code=1100, message=failed to create query plan: cannot parse expression: node_name IN [ACTG1,AIF1,ALOX5AP], error: field ACTG1 not exist: invalid parameter)>, <Time:{'RPC start': '2025-07-09 17:58:25.105382', 'RPC error': '2025-07-09 17:58:25.106509'}> (decorators.py:140)


MilvusException: <MilvusException: (code=1100, message=failed to create query plan: cannot parse expression: node_name IN [ACTG1,AIF1,ALOX5AP], error: field ACTG1 not exist: invalid parameter)>

In [ ]:
# Vector similarity search in Milvus
vector_to_search = nodes_df["feat_emb"].iloc[0]
search_params = {"metric_type": "IP"}
results = collection.search(
    data=[vector_to_search],
    anns_field="feat_emb",
    param=search_params,
    limit=10,
    output_fields=["node_id", "node_name"]
)
results

In [ ]:
for node_type, group in multimodal_df.groupby("q_node_type"):
    print(f"Processing node type: {node_type}")
    
    # Load the collection
    collection = Collection(name=f"{milvus_database}_nodes_{node_type.replace('/', '_')}")
    collection.load()

    # Query the collection with node names from multimodal_df
    q_node_names =  getattr(multimodal_df['q_node_name'], 
                            "to_pandas", 
                            lambda: multimodal_df['q_node_name'])().tolist()
    q_columns = ["node_id", "node_type", "feat", "feat_emb", "desc", "desc_emb"]
    res = collection.query(
        expr=f'node_name IN [{','.join(f'"{name}"' for name in q_node_names)}]',
        output_fields=q_columns,
    )
    
    # Convert the result to a DataFrame
    res_df = df.DataFrame(res)[q_columns]
    


In [ ]:
res_df

In [ ]:
[r['node_id'] for r in res]

In [ ]:
[r['node_type'] for r in res]

In [ ]:
[r['desc'] for r in res]

In [ ]:
multimodal_df['q_node_name']

In [ ]:
multimodal_df[['q_node_name']].to_csv(header=False, index=False, sep=",")

In [ ]:
getattr(multimodal_df['q_node_name'], "to_pandas", lambda: multimodal_df['q_node_name'])().tolist()

In [ ]:
milvus

In [48]:
# Check if mapping_edges and mapping_edges_new are identical
mapping_edges = {int(i): int(j) for i, j in enumerate(real_["indices"])}
mapping_edges_new = dict(zip(range(len(real_["indices"])), real_["indices"].tolist()))

identical = mapping_edges == mapping_edges_new
print(f"Are both dictionaries identical? {identical}")

# Optionally, print differences if not identical
if not identical:
    diff_keys = set(mapping_edges.keys()) ^ set(mapping_edges_new.keys())
    print(f"Different keys: {diff_keys}")
    for k in mapping_edges.keys():
        if k in mapping_edges_new and mapping_edges[k] != mapping_edges_new[k]:
            print(f"Value mismatch at key {k}: {mapping_edges[k]} != {mapping_edges_new[k]}")

Are both dictionaries identical? True
